In [5]:
# Imports and Dataset Loading
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load data (userID, animeID, rating)
df = pd.read_csv('../data/ratings.csv')
# Load data (animeID, title, alternative_title, type, year, score, episodes, mal_url, sequel, image_url, genres, genres_detailed)
#animes = pd.read_csv('../data/animes.csv')
print(df.head())
print("-------------------------")
#print(animes.head())

   userID  animeID  rating
0       1        1      10
1       1        2      10
2       1        3       7
3       1        4      10
4       1        5      10
-------------------------


In [17]:
df.shape

(148170496, 3)

In [13]:
df['user_idx'], self.user_ids = pd.factorize(df['userID'])
df['anime_idx'], self.anime_ids = pd.factorize(df['animeID'])
head(df)

MemoryError: Unable to allocate 1.10 GiB for an array with shape (148170496,) and data type int64

In [8]:
train_df, test_df = train_test_split(
            df,
            test_size=0.25,
            random_state=42,
            stratify=df['userID']
        )
train_matrix = train_df.pivot_table(
            index='userID', columns='animeID', values='rating'
        ).fillna(0)

MemoryError: Unable to allocate 833. MiB for an array with shape (109128738,) and data type int64

In [ ]:
# === 2. Create User-Item Matrix ===
ratings_matrix = df.pivot_table(index='user_id', columns='anime_id', values='rating')
ratings_matrix.fillna(0, inplace=True)

# Optional: show matrix shape
print("Matrix shape:", ratings_matrix.shape)

# === 3. Apply Truncated SVD (Matrix Factorization) ===
svd = TruncatedSVD(n_components=20, random_state=42)
matrix_reduced = svd.fit_transform(ratings_matrix)

# Reconstructed ratings approximation
reconstructed_ratings = np.dot(matrix_reduced, svd.components_)

# Convert back to DataFrame
predicted_df = pd.DataFrame(reconstructed_ratings, index=ratings_matrix.index, columns=ratings_matrix.columns)

# === 4. Load New User's Ratings ===
new_user_df = pd.read_csv('../data/new_user.csv')  # columns: anime_id, rating

# Create a pseudo user vector with 0s
new_user_vector = pd.Series(0, index=ratings_matrix.columns)

# Fill in their known ratings
for _, row in new_user_df.iterrows():
    if row['anime_id'] in new_user_vector.index:
        new_user_vector[row['anime_id']] = row['rating']

# Project the new user using the same SVD
new_user_proj = svd.transform([new_user_vector])[0]
new_user_pred_ratings = np.dot(new_user_proj, svd.components_)

# === 5. Recommend Top N Unseen Anime ===
watched_anime = new_user_df['anime_id'].tolist()

# Predict only for unseen anime
anime_ids = ratings_matrix.columns
recommendations = []
for idx, anime_id in enumerate(anime_ids):
    if anime_id not in watched_anime:
        recommendations.append((anime_id, new_user_pred_ratings[idx]))

top_n = sorted(recommendations, key=lambda x: x[1], reverse=True)[:10]

print("\n🎯 Top 10 Recommended Anime IDs:")
for anime_id, score in top_n:
    print(f"Anime ID: {anime_id}, Score: {score:.2f}")